# 3F7 Lab: CamZIP

## Tree data structures

This lab has been modified from previous versions to define all functions inline instead of importing them from a library, in response to student feedback that rightly pointed out that Jupyter Notebooks don't work well with imported libraries if you have to edit and modify the library (essentially you have to restart the Jupyter engine every time and re-run all the relevant cells...) The drawback now is that some cells will look very busy as we define all the auxiliary functions inline.

We put together a number of tools for handling trees in the 3F7 lab. You do not need to examine the details in this cell.

In [2]:
## Please ignore the details of this cell, it contains in-line  library function definitions

def tree2newick(t, labels = []):
    return(xtree2newick(tree2xtree(t, labels)))

def xtree2newick(xt, labels = [], n = -1):
    """
    Converts an extended tree to Newick format.

    Parameters:
    -----------
    xt: list of lists
    Extended tree defined as list of lists where each list contains as its
    first element a pointer to its parent and the second element is a list
    containing pointers its children

    n: int
    USED INTERNALLY FOR RECURSION, DO NOT SET!

    Returns:
    --------
    string
    Tree description in Newick format (can be used to view a tree viewing standard
    tools, e.g., phylo.io online phylogenetic tree viewer.)

    Written by Jossy, 2018
    """

    if len(labels) == 0:
        try:
            labels = [int(a[2]) for a in xt]
            if max(labels) < 128:
                labels = [chr(a) for a in labels]
        except ValueError:
            labels = [a[2] for a in xt]
        for k in range(len(labels)):
            if labels[k] == ',':
                labels[k] = 'comma'
            elif labels[k] == '(':
                labels[k] = 'left parenthesis'
            elif labels[k] == ')':
                labels[k] = 'right parenthesis'
            elif labels[k] == '\n':
                labels[k] = 'carriage return'
            elif labels[k] == '|':
                labels[k] = 'vertical bar'
            elif labels[k] == ':':
                labels[k] = 'colon'
            elif labels[k] == ';':
                labels[k] = 'semi-colon'
            elif labels[k] == ' ':
                labels[k] = 'space'
            elif labels[k] == '[':
                labels[k] = 'left square bracket'
            elif labels[k] == ']':
                labels[k] = 'right square bracket'

    if (n == -1): # find root and set n to index of root
        n = [ind for ind in range(len(xt)) if xt[ind][0] == -1]
        if len(n) != 1:
            raise NameError('Tree with no root or several roots')
        n = n[0] # n is a list with 1 element, retrieve that element

    # find the children of the current node
    children = [a for a in xt[n][1] if a != -1]
    # if the current node has no children, return its label
    if len(children) == 0:
        return '%(myname)s' % {'myname': labels[n]}
    # for every child, recurse the function and separate its outcomes by commas
    outstring = xtree2newick(xt,labels,children[0]) # first child
    for k in range(1,len(children)): # remaining children
        outstring = outstring + ',' + xtree2newick(xt,labels,children[k])

    # surround the string by parenthesis and append the current node label
    outstring = '(' + outstring + ')%(myname)s' % {'myname': labels[n]}
    return outstring # return the resulting string


def tree2xtree(t, labels = []):
    xt = [[] for node in t]
    for node in range(len(t)):
        children = [ind for ind in range(len(t)) if t[ind] == node]
        xt[node].append(t[node])
        xt[node].append(children)

    # if tree only partially labeled or no labels, use partial labels
    # and natural numbering for remaining nodes, starting from leaves first
    if len(labels) < len(t):
        xtlabels = [[] for k in range(len(t))]
        leavesfirst = [k for k in range(len(xt)) if len(xt[k][1]) == 0]
        leavesfirst.extend([k for k in range(len(xt)) if len(xt[k][1]) > 0])
        for k in range(len(labels)):
            xtlabels[leavesfirst[k]] = labels[k]
        for k in range(len(labels),len(xt)):
            xtlabels[leavesfirst[k]] = str(k+1000)
        labels = xtlabels
    for node in range(len(xt)):
        xt[node].append(labels[node])

    return xt

def xtree2tree(xt):
    return [node[0] for node in xt]

def xtree2code(xt):
    leaves = [ind for ind in range(len(xt)) if len(xt[ind][1]) == 0]
    code = {}
    for leaf in leaves:
        codeword = []
        node = leaf
        while xt[node][0] != -1: # while node is not root
            parent = xt[node][0] # node's parent
            # which number child am I?
            nchild = [ind for ind in range(len(xt[parent][1])) if xt[parent][1][ind] == node]
            codeword.insert(0, nchild[0])
            node = parent
        code[xt[leaf][2]] = codeword

    return code


def tree2code(t, labels=[]):
    return xtree2code(tree2xtree(t, labels))

def code2xtree(c):
    xt = [[-1, []]] # init tree with just a root
    for symbol in c:
        node = 0 # reset to root
        for digit in c[symbol]:
            while len(xt[node][1]) <= digit:
                xt[node][1].append(-1)
            if xt[node][1][digit] == -1:
                # create new node
                xt.append([node, []])
                xt[node][1][digit] = len(xt)-1
            node = xt[node][1][digit]
        xt[node].append(symbol)

    # label the nodes that are not codewords by numbering them
    not_codeword_nodes = [k for k in range(len(xt)) if len(xt[k]) < 3]
    for k in range(len(not_codeword_nodes)):
        xt[not_codeword_nodes[k]].append(str(k))

    return xt


def code2tree(c):
    return xtree2tree(code2xtree(c))


Now define a simple tree (play around with this command and construct more complicated trees....)

In [3]:
t = [-1,0,1,1,0]

The following command will print a string that can be copy-pasted into a tree visualising website like [phylo.io](https://phylo.io) (don't forget to add a new line at the end of the string after cutting and pasting)

In [4]:
print(tree2newick(t))
print('Cut and paste the string on the previous line and add a "new line" at the end of the string.')

((1000,1001)1004,1002)1003
Cut and paste the string on the previous line and add a "new line" at the end of the string.


You can also add labels to the nodes in the `tree2newick` command.

In [5]:
print(tree2newick(t,['root', 'child 0', 'grandchild 0', 'grandchild 1', 'child 1']))

((grandchild 0,grandchild 1)child 0,child 1)root


If there are less labels than nodes, the labels will be interpreted "leaves first"

In [6]:
print(tree2newick(t,['symbol 0','symbol 1', 'symbol 2']))

((symbol 0,symbol 1)1004,symbol 2)1003


The following command converts a variable-length code described by a tree to a code table format.

In [7]:
print(tree2code(t))

{'1000': [0, 0], '1001': [0, 1], '1002': [1]}


Verify that the inverse function can recover the tree.

In [8]:
print(code2tree(tree2code(t)))

[-1, 0, 1, 1, 0]


But the following may happen as well. Can you explain why?

In [9]:
print(code2tree(tree2code([3,3,4,4,-1])))

[-1, 0, 1, 1, 0]


In [10]:
print(tree2newick([3,3,4,4,-1], ['grandchild 0', 'grandchild 1', 'child 0', 'child 1', 'root']))

(child 0,(grandchild 0,grandchild 1)child 1)root


Similarly but far more problematic is the following inversion. The resulting assignment of codeword to symbols is fundamentally different from the original and would result in wrong decoding.

In [11]:
print(tree2code(code2tree({'0':[1], '1':[0,1], '2':[0,0,1], '3':[0,0,0]})))

{'1000': [0], '1001': [1, 0], '1002': [1, 1, 0], '1003': [1, 1, 1]}


These problems are all solved when using the extended tree format.

In [12]:
xt = tree2xtree([3,3,4,4,-1], ['a', 'b', 'c'])
print(xt)

[[3, [], 'a'], [3, [], 'b'], [4, [], 'c'], [4, [0, 1], '1003'], [-1, [2, 3], '1004']]


In [13]:
print(xtree2code(code2xtree({'0':[1], '1':[0,1], '2':[0,0,1], '3':[0,0,0]})))

{'0': [1], '1': [0, 1], '2': [0, 0, 1], '3': [0, 0, 0]}


## Testing your Shannon-Fano Code

The following cell contains a partial implementation of the Shannon Fano code. Some commands are missing and you must complete the function definition before you can proceed to the subsequent cells.

In [14]:
## The function definition below is INCOMPLETE. You must complete it before you
## can proceed to the subsequent commands in this notebook.

from math import log2, ceil

def shannon_fano(p):

    # Begin by sorting the probabilities in decreasing order, as required
    # in Shannon's paper.
    p = dict(sorted([(a,p[a]) for a in p if p[a]>0.0], key = lambda el: el[1], reverse = True))

    # Compute the cumulative probability distribution
    # this can be done easily with numpy.cumsum but we will do it by hand. Note
    # that this is not a time-critical operation so efficiency is not an issue.

    # step 1: initialise f to be a list with one element 0 (this is because Shannon
    # requires the cumulative probability to be the sum up to AND EXCLUDING the current
    # symbol, so the first element of f should be zero.

    # ... (whenever you see "..." you are expected to complete a missing command

    # step 2: compute the runninng sum
    for a in p: # for every probability in p
        # you now want to append to f the sum of its last entry (which you can access as [-1])
        # and the probability p[a] of the current symbol

        # ...

    # the resulting cumulative has one too many element at the end, the sum of all probabilities
    # that should equal to one. You can use the "pop" command to delete the last element in a list.

    # ...

    # We now convert the list you computed into a dictionary
    f = dict([(a,mf) for a,mf in zip(p,f)])

    # assign the codewords
    code = {} # initialise as an empty dictionary
    for a in p: # for each probability

        # Compute the codeword length according to the Shannon-Fano formula
        # you want to use the functions "ceil()" and "log2()" we imported
        # from the math library
        #...
        # (assign variable name "length")

        codeword = [] # initialise current codeword
        myf = f[a]
        # for each position in length, we will multiply myf by 2 and take the
        # integral part as our binary digit
        for pos in range(length):
            # multiply myf by 2 (shifting it "left" in binary)
            #...

            # if the resulting myf is larger than 1, append a 1 to the codeword,
            # whereas if it is smaller than 1 you should append a 0
            # If it is larger than 1, you sould also substratct 1 from myf.
            #...
            #...
            #...
            #...
            #...
        code[a] = codeword # assign the codeword

    return code # return the code table


This next section can only be completed once you have a working Shannon-Fano function `shannon_fano()`

In [15]:
from random import random
p = [random() for k in range(16)]
p = dict([(chr(k+ord('a')),p[k]/sum(p)) for k in range(len(p))])
print(f'Probability distribution: {p}\n')
c = shannon_fano(p)
print(f'Codebook: {c}\n')
xt = code2xtree(c)
print(f'Cut and paste for phylo.io: {xtree2newick(xt)}')

Probability distribution: {'a': 0.07939477952522697, 'b': 0.05962233625893101, 'c': 0.07336844674948974, 'd': 0.08380502566806766, 'e': 0.043354563675279324, 'f': 0.08788127339828825, 'g': 0.004378295319653532, 'h': 0.08000879894205384, 'i': 0.08213842487272209, 'j': 0.018332709845997223, 'k': 0.036605311387913786, 'l': 0.00033384066431685745, 'm': 0.0369487561215596, 'n': 0.08327675733023814, 'o': 0.1137568136951259, 'p': 0.11679386654513582}

Codebook: {'p': [0, 0, 0, 0], 'o': [0, 0, 0, 1], 'f': [0, 0, 1, 1], 'd': [0, 1, 0, 1], 'n': [0, 1, 1, 0], 'i': [0, 1, 1, 1], 'h': [1, 0, 0, 1], 'a': [1, 0, 1, 0], 'c': [1, 0, 1, 1], 'b': [1, 1, 0, 0, 1], 'e': [1, 1, 0, 1, 1], 'm': [1, 1, 1, 0, 0], 'k': [1, 1, 1, 1, 0], 'j': [1, 1, 1, 1, 1, 0], 'g': [1, 1, 1, 1, 1, 1, 1, 0], 'l': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]}

Cut and paste for phylo.io: ((((p,o)3,(f)4)2,((d)6,(n,i)7)5)1,(((h)10,(a,c)11)9,(((b)14,(e)15)13,((m)17,(k,(j,((g,((((l)25)24)23)22)21)20)19)18)16)12)8)0


We can upload data from a file, for example `hamlet.txt`, and display the first few lines...

Before you execute the next cell, make sure `hamlet.txt` is accessible: if using this Jupyter notebook on your computer, make sure the file is in the same director. If you using this Jupyter notebook on Google Colabs, click on the folder symbol on the left and upload the file to the session folder (the file gets deleted if you start a new session so you might have to do this every time.)

In [17]:
f = open('hamlet.txt', 'r')
hamlet = f.read()
f.close()
print(hamlet[:294])

        HAMLET


        DRAMATIS PERSONAE


CLAUDIUS        king of Denmark. (KING CLAUDIUS:)

HAMLET  son to the late, and nephew to the present king.

POLONIUS        lord chamberlain. (LORD POLONIUS:)

HORATIO friend to Hamlet.

LAERTES son to Polonius.

LUCIANUS        nephew to the king.


We now compute the startistics of the file:

In [18]:
from itertools import groupby
frequencies = dict([(key, len(list(group))) for key, group in groupby(sorted(hamlet))])
Nin = sum([frequencies[a] for a in frequencies])
p = dict([(a,frequencies[a]/Nin) for a in frequencies])
print(f'File length: {Nin}')

File length: 207039


We can view the alphabet of symbols used in the file:

In [19]:
print(list(p))
print(len(p))

['\n', ' ', '!', '&', "'", '(', ')', ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|']
67


We are now ready to construct the Shannon-Fano code for this file, and view its tree (cut and paste into [phylo.io](https://phylo.io), don't forget to add a carriage return at the end, click on "Branch Labels/Support" under "Settings", then right-click on the root of the tree and select "expand all".

In [20]:
c = shannon_fano(p)
print(len(c))
xt = code2xtree(c)
print(len(xt))
print(xtree2newick(xt))
print(len(c['&']))

67
166
((space,((e,(t)4)3,((o)6,(a,s)7)5)2)1,((((n)11,(h,i)12)10,((r,(carriage return)15)14,((l)17,(d)18)16)13)9,((((u,m)22,(comma,(y)24)23)21,(((w)27,(f)28)26,((c,g)30,(p)31)29)25)20,(((((A)36,(b,T)37)35,((I)39,(E)40)38)34,(((.)43,(L)44)42,((v)46,(k,O)47)45)41)33,((((')51,(H,R)52)50,((N,(S)55)54,((U)57,(M)58)56)53)49,((((semi-colon,colon)62,(D)63)61,((C,G)65,(W,?)66)64)60,(((-,(P)70)69,((!)72,(left square bracket,right square bracket)73)71)68,(((B,F)76,((x,K)78)77)75,(((q)81,(Y,(j)83)82)80,(((Q)86,(Z)87)85,((z,(vertical bar)90)89,((V)92,((left parenthesis,right parenthesis)94,((J)96,((&)98)97)95)93)91)88)84)79)74)67)59)48)32)19)8)0
16


The following defines a number of library functions to encode and decode variable length codes using a code table/tree. You can ignore the details of these programmes.

In [21]:
## Please ignore the details of this cell, it contains in-line  library function definitions


def bits2bytes(x):
    n = len(x)+3
    r = (8 - n % 8) % 8
    prefix = format(r, '03b')
    x = ''.join(str(a) for a in x)
    suffix = '0'*r
    x = prefix + x + suffix
    x = [x[k:k+8] for k in range(0,len(x),8)]
    y = []
    for a in x:
        y.append(int(a,2))

    return y

def bytes2bits(y):
    x = [format(a, '08b') for a in y]
    r = int(x[0][0:3],2)
    x = ''.join(x)
    x = [int(a) for a in x]
    for k in range(3):
        x.pop(0)
    for k in range(r):
        x.pop()
    return x


def vl_encode(x, c):
    y = []
    for a in x:
        y.extend(c[a])
    return y


def vl_decode(y, xt):
    x = []
    root = [k for k in range(len(xt)) if xt[k][0]==-1]
    if len(root) != 1:
        raise NameError('Tree with no or multiple roots!')
    root = root[0]
    leaves = [k for k in range(len(xt)) if len(xt[k][1]) == 0]

    n = root
    for k in y:
        if len(xt[n][1]) < k:
            raise NameError('Symbol exceeds alphabet size in tree node')
        if xt[n][1][k] == -1:
            raise NameError('Symbol not assigned in tree node')
        n = xt[n][1][k]
        if len(xt[n][1]) == 0: # it's a leaf!
            x.append(xt[n][2])
            n = root
    return x

Now we can actually encode the file `hamlet.txt` using the Shannon-Fano code we constructed.

In [22]:
hamlet_sf = vl_encode(hamlet, c)
print(f'Length of binary sequence: {len(hamlet_sf)}')

Length of binary sequence: 997548


We have commands to convert a bit sequence into a byte sequence (including a 3 bit prefix that helps us determine the length of the bit sequence):

In [23]:
x = bits2bytes([0,1])
print([format(a, '08b') for a in x])
y = bytes2bits(x)
print(f'The original bits are: {y}')
print(bits2bytes([0,1,1,0,1,1,0,0,0]))

['01101000']
The original bits are: [0, 1]
[141, 128]


We now apply the bits to byte conversion to the compressed text of Hamlet to compute the length of the compressed file.

In [24]:
hamlet_zipped = bits2bytes(hamlet_sf)
Nout = len(hamlet_zipped)
print(f'Length of compressed string: {Nout}')

Length of compressed string: 124694


The compression ratio can be expressed in two ways, unitless or in bits/bytes:

In [25]:
print(f'Compression ratio (rateless): {Nout/Nin}')
print(f'Compression ratio (bits per byte): {8.0*Nout/Nin}')

Compression ratio (rateless): 0.6022730017049928
Compression ratio (bits per byte): 4.818184013639942


The lower bound for compression is the Entropy, measured in bits, that can be computed using an in-line function in Python:

In [26]:
from math import log2
H = lambda pr: -sum([pr[a]*log2(pr[a]) for a in pr])
print(f'Entropy: {H(p)}')

Entropy: 4.449863631694343


We now proceed to decode the compressed Hamlet sequence

In [27]:
xt = code2xtree(c)
hamlet_unzipped = vl_decode(hamlet_sf, xt)
print(f'Length of unzipped file: {len(hamlet_unzipped)}')

Length of unzipped file: 207039


We can view the first few lines of the input (note the command `join` that turns the list of strings into one string)

In [28]:
print(''.join(hamlet_unzipped[:294]))

        HAMLET


        DRAMATIS PERSONAE


CLAUDIUS        king of Denmark. (KING CLAUDIUS:)

HAMLET  son to the late, and nephew to the present king.

POLONIUS        lord chamberlain. (LORD POLONIUS:)

HORATIO friend to Hamlet.

LAERTES son to Polonius.

LUCIANUS        nephew to the king.


## Compressing and uncompressing files

This is where we put it all together, compressing directly from input to output file. Play around with these commands once you implemented Huffman coding and arithmetic coding. We begin by defining the compression and decompression library functions. You can ignore the details of these function definitions.

In [29]:
## Please ignore the details of this cell, it contains in-line  library function definitions

from json import dump,load
from sys import argv,exit

def camzip(method, filename):

    with open(filename, 'rb') as fin:
        x = fin.read()

    frequencies = dict([(key, len(list(group))) for key, group in groupby(sorted(x))])
    n = sum([frequencies[a] for a in frequencies])
    p = dict([(a,frequencies[a]/n) for a in frequencies])

    if method == 'huffman' or method == 'shannon_fano':
        if (method == 'huffman'):
            xt = huffman(p)
            c = xtree2code(xt)
        else:
            c = shannon_fano(p)
            xt = code2xtree(c)

        y = vl_encode(x, c)

    elif method == 'arithmetic':
        y = arithmetic.encode(x,p)

    else:
        raise NameError('Compression method %s unknown' % method)


    y = bytes(bits2bytes(y))

    outfile = filename + '.cz' + method[0]

    with open(outfile, 'wb') as fout:
        fout.write(y)

    pfile = filename + '.czp'
    n = len(x)

    with open(pfile, 'w') as fp:
        dump(frequencies, fp)


def camunzip(filename):
    if (filename[-1] == 'h'):
        method = 'huffman'
    elif (filename[-1] == 's'):
        method = 'shannon_fano'
    elif (filename[-1] == 'a'):
        method = 'arithmetic'
    else:
        raise NameError('Unknown compression method')

    with open(filename, 'rb') as fin:
        y = fin.read()
    y = bytes2bits(y)

    pfile = filename[:-1] + 'p'
    with open(pfile, 'r') as fp:
        frequencies = load(fp)
    n = sum([frequencies[a] for a in frequencies])
    p = dict([(int(a),frequencies[a]/n) for a in frequencies])

    if method == 'huffman' or method == 'shannon_fano':
        if (method == 'huffman'):
            xt = huffman(p)
            c = xtree2code(xt)
        else:
            c = shannon_fano(p)
            xt = code2xtree(c)

        x = vl_decode(y, xt)

    elif method == 'arithmetic':
        x = arithmetic.decode(y,p,n)

    else:
        raise NameError('This will never happen (famous last words)')

    # '.cuz' for Cam UnZipped (don't want to overwrite the original file...)
    outfile = filename[:-4] + '.cuz'

    with open(outfile, 'wb') as fout:
        fout.write(bytes(x))


The next commands define the method to be used and the filename. Modify those when you are trying other methods on various files.

In [30]:
method = 'shannon_fano'
filename = 'hamlet.txt'

Now we do the actual compression and decompression...

In [31]:
camzip(method, filename)
camunzip(filename + '.cz' + method[0])

The next few lines perform various statistical measurements and verifies that the decompressed file is identical to the compressed file.

In [32]:
from filecmp import cmp
from os import stat
from json import load
Nin = stat(filename).st_size
print(f'Length of original file: {Nin} bytes')
Nout = stat(filename + '.cz' + method[0]).st_size
print(f'Length of compressed file: {Nout} bytes')
print(f'Compression rate: {8.0*Nout/Nin} bits/byte')
with open(filename + '.czp', 'r') as fp:
    freq = load(fp)
pf = dict([(a, freq[a]/Nin) for a in freq])
print(f'Entropy: {H(pf)} bits per symbol')
if cmp(filename,filename+'.cuz'):
    print('The two files are the same')
else:
    print('The files are different')

Length of original file: 207039 bytes
Length of compressed file: 124694 bytes
Compression rate: 4.818184013639942 bits/byte
Entropy: 4.449863631694343 bits per symbol
The two files are the same


## Huffman coding

The following cell contains a partial implementation of the Huffman code. Some commands are missing and you must complete the function definition before you can proceed to the subsequent cells.

In [33]:
## The function definition below is INCOMPLETE. You must complete it before you
## can proceed to the subsequent commands in this notebook.

def huffman(p):
    # create an xtree with all the source symbols (to be the leaves) initially orphaned
    xt = [[-1,[], a] for a in p]
    # label the probabilities with a "pointer" to their corresponding nodes in the tree
    # in the process, we convert the probability vector from a Python dictionary to a
    # list of tuples (so we can modify it)
    p = [(k,p[a]) for k,a in zip(range(len(p)),p)]

    # the leaves are labeled according to the symbols in the probability vector. We will
    # label the remaining tree nodes (mainly for visualisation purposes) with numbers
    # starting at len(p)
    nodelabel = len(p)

    # this loop will gradually increase the tree and reduce the probability list.
    # It will run until there is only one probability left in the list (which probability
    # will by then be 1.0)
    while len(p) > 1:
        # sort probabilities in ascending order (so [0] and [1] are smallest)
        # using the command "sorted" and, as its second element, the expression
        #  "key = lambda el:el[1]" (which is an inline function to retrieve the
        # second entry from a tuple.) Note that the natural order of "sorted" is
        # increasing, which is as you want it in this case.
        # The output of sorted() can be written back to p (i.e. p = sorted(p, ....))
        #...

        # Now append a new node to the tree xt with no parent (parent = -1),
        # no children (children = []) and label str(nodelabel)
        #...

        # we incrase the variable nodelabel by 1 for its next use
        nodelabel += 1

        # assign parent of the nodes pointed to by the smallest probabilities
        # Note that the smallest probabilities are now p[0] and p[1], so their
        # pointers to nodes in xt are p[0][0] and p[1][0]. The corresponding
        # xt nodes should be assigned the new node you created as a parent,
        # whose index is len(xt)-1 since it has been appended at the end of xt
        #...
        #...

        # assign the children of new node to be the nodes pointed to by
        # p[0] and p[1]. Note that the new node can be addressed as xt[-1]
        #...

        # create a new entry pointing to the new node in the list of probabilities
        # This new entry should be a tuple with len(xt)-1 as its first element,
        # and the sum of the probabilities in p[0] and p[1] as its second element
        #...

        # remove the two nodes with the smallest probability
        p.pop(0)
        p.pop(0)
        # (using pop(0) twice removes the first element of the list twice
        # and hence removes the first 2 elements.)

    return(xt)





This section will only work once you have a working function `huffman()`. We first repeat the tree construction and visualisation.

In [34]:
xt = huffman(p)
print(xtree2newick(xt))

((((a,(((v,L)96,c)105,d)113)121,(((f,((M,(P,((Z,(V,(right parenthesis,((&,J)67,left parenthesis)68)69)70)72,x)76)81)88,.)97)106,l)114,o)122)127,((t,((w,((U,S)89,E)98)107,carriage return)115)123,((((I,T)99,y)108,((b,A)100,comma)109)116,(((((((Q,(vertical bar,z)71)73,(j,Y)74)77,-)82,N)90,p)101,(((?,W)83,R)91,((G,(F,B)78)84,H)92)102)110,r)117)124)128)130,(space,((e,(i,(m,((',(C,(left square bracket,right square bracket)79)85)93,g)103)111)118)125,((h,n)119,(s,(u,(((D,colon)86,O)94,(k,(((q,K)75,!)80,semi-colon)87)95)104)112)120)126)129)131)132


In [35]:
xtree2code(huffman({'a':.5, 'b':.25, 'c':.25, 'd':0}))

{'a': [0], 'b': [1, 1, 1], 'c': [1, 0], 'd': [1, 1, 0]}

Observe how the Huffman tree differs from the Shannon-Fano tree. What are its shortest and its longest codeword? You can use the `camzip` code above changing the method to `'huffman'` to test the compression rate etc. You may also want to do it by hand to test the error resilience:

In [36]:
c = xtree2code(xt)
hamlet_huf = vl_encode(hamlet, c)
hamlet_decoded = vl_decode(hamlet_huf, xt)
print(''.join(hamlet_decoded[:294]))

        HAMLET


        DRAMATIS PERSONAE


CLAUDIUS        king of Denmark. (KING CLAUDIUS:)

HAMLET  son to the late, and nephew to the present king.

POLONIUS        lord chamberlain. (LORD POLONIUS:)

HORATIO friend to Hamlet.

LAERTES son to Polonius.

LUCIANUS        nephew to the king.


We now introduce a random bit flip (bit 400 flipped) in the compressed sequence and observe the result.

In [37]:
hamlet_corrupted = hamlet_huf.copy()
hamlet_corrupted[400] ^= 1
hamlet_decoded = vl_decode(hamlet_corrupted, xt)
print(''.join(hamlet_decoded[:297]))

        HAMLET


        DRAMATIS PERSONAE


CLAUDIUS        king of Denmark. y,; 
aNG CLAUDIUS:)

HAMLET  son to the late, and nephew to the present king.

POLONIUS        lord chamberlain. (LORD POLONIUS:)

HORATIO friend to Hamlet.

LAERTES son to Polonius.

LUCIANUS        nephew to the king.


## Arithmetic coding

We first try "by hand" to operate the steps of arithmetic coding using floating point numbers. We first compute the cumulative probability distribution.

In [38]:
f = [0.0]
for a in p:
    f.append(f[-1]+p[a])
f.pop()
f = dict([(a,f[k]) for a,k in zip(p,range(len(p)))])

We now perform by hand the first `n=4` steps of arithmetic coding. Vary `n` to observe the loss of precision.

In [39]:
lo, hi = 0.0, 1.0
n = 4
for k in range(n):
    a = hamlet[k]
    lohi_range = hi - lo
    hi = lo + lohi_range * (f[a] + p[a])
    lo = lo + lohi_range * f[a]
print(f'lo = {lo}, hi = {hi}, hi-lo = {hi-lo}')

lo = 0.03994572862162076, hi = 0.04551184792510039, hi-lo = 0.005566119303479632


The output sequence is roughly the binary expression of `lo` (not exactly) and we can compute and observe it. What length `ell` would we need when encoding all of Hamlet?

In [40]:
from math import floor, ceil
ell = ceil(-log2(hi-lo))+2 if hi-lo > 0.0 else 96
print(bin(floor(lo*2**ell)))

0b101000


The cell below contains a skeleton arithmetic encoder that you need to complete before executing the commands below. They won't work until you've completed the implementation of the arithmetic encoder. Following that, there will be a cell containing the definition of an arithmetic decoder. The implementation of the decoder is complete and does not need to be amended (though you may want to look at some of the details in the programme to help you complete the encoder as there are similarities between the two.

In [41]:
## The function definition below is INCOMPLETE. You must complete it before you
## can proceed to the subsequent commands in this notebook.

from sys import stdout as so
from bisect import bisect

def encode(x, p):

    precision = 32
    one = int(2**precision - 1)
    quarter = int(ceil(one/4))
    half = 2*quarter
    threequarters = 3*quarter

    p = dict([(a,p[a]) for a in p if p[a]>0])

    # Compute cumulative probability as in Shannon-Fano but WITHOUT
    # sorting the probability distrbution
    # ...
    # ...
    # ...
    # ...
    f = dict([(a,mf) for a,mf in zip(p,f)])

    y = [] # initialise output list
    lo,hi = 0,one # initialise lo and hi to be [0,1.0)
    straddle = 0 # initialise the straddle counter to 0


    for k in range(len(x)): # for every symbol

        # arithmetic coding is slower than vl_encode, so we display a "progress bar"
        # to let the user know that we are processing the file and haven't crashed...
        if k % 100 == 0:
            so.write('Arithmetic encoded %d%%    \r' % int(floor(k/len(x)*100)))
            so.flush()

        # 1) calculate the interval range to be the difference between hi and lo and
        # add 1 to the difference. The added 1 is necessary to avoid rounding issues
        # lohi_range = ....

        # 2) narrow the interval end-points [lo,hi) to the new range [f,f+p]
        # within the old interval [lo,hi], being careful to round 'innwards' so
        # the code remains prefix-free (you want to use the functions ceil and
        # floor). This will require two instructions. Note that we start computing
        # the new 'lo', then compute the new 'hi' using the scaled probability as
        # the offset from the new 'lo' to the new 'hi'
        # ...
        # ...

        if (lo == hi):
            raise NameError('Zero interval!')

        # Now we need to re-scale the interval if its end-points have bits in common,
        # and output the corresponding bits where appropriate. We will do this with an
        # infinite loop, that will break when none of the conditions for output / straddle
        # are fulfilled
        while True:
            if hi < half: # if lo < hi < 1/2
                # stretch the interval by 2 and output a 0 followed by 'straddle' ones (if any)
                # and zero the straddle after that. In fact, HOLD OFF on doing the stretching:
                # we will do the stretching at the end of the if statement
                # ...  append a zero to the output list y
                # ...  extend by a sequence of 'straddle' ones
                # ...  zero the straddle counter
            elif lo >= half: # if hi > lo >= 1/2
                # stretch the interval by 2 and substract 1, and output a 1 followed by 'straddle'
                # zeros (if any) and zero straddle after that. Again, HOLD OFF on doing the stretching
                # as this will be done after the if statement, but note that 2*interval - 1 is equivalent
                # to 2*(interval - 1/2), so for now just substract 1/2 from the interval upper and lower
                # bound (and don't forget that when we say "1/2" we mean the integer "half" we defined
                # above: this is an integer arithmetic implementation!
                # ...  append a 1 to the output list y
                # ...  extend 'straddle' zeros
                # ...  reset the straddle counter
                # ...
                # ...  substract half from lo and hi
            elif lo >= quarter and hi < threequarters: # if 1/4 < lo < hi < 3/4
                # we can increment the straddle counter and stretch the interval around
                # the half way point. This can be impemented again as 2*(interval - 1/4),
                # and as we will stretch by 2 after the if statement all that needs doing
                # for now is to subtract 1/4 from the upper and lower bound
                # ...  increment straddle
                # ...
                # ...  subtract 'quarter' from lo and hi
            else:
                break # we break the infinite loop if the interval has reached an un-stretchable state
            # now we can stretch the interval (for all 3 conditions above) by multiplying by 2
            # ...  multiply lo by 2
            # ...  multiply hi by 2 and add 1 (I DON'T KNOW WHY +1 IS NECESSARY BUT IT IS. THIS IS MAGIC.
            #      A BOX OF CHOCOLATES FOR ANYONE WHO GIVES ME A WELL ARGUED REASON FOR THIS... It seems
            #      to solve a minor precision problem.)

    # termination bits
    # after processing all input symbols, flush any bits still in the 'straddle' pipeline
    straddle += 1 # adding 1 to straddle for "good measure" (ensures prefix-freeness)
    if lo < quarter: # the position of lo determines the dyadic interval that fits
        # ...  append a zero followed by "straddle" ones
        # ...
    else:
        # ...  append a 1 followed by "straddle" zeros


    return(y) # return the final output string

And now for the decoder (complete definition, no editing needed):

In [42]:
## The function definition below is COMPLETE. You do not need to edit it.

def arith_decode(y,p,n):
    precision = 32
    one = int(2**precision - 1)
    quarter = int(ceil(one/4))
    half = 2*quarter
    threequarters = 3*quarter

    p = dict([(a,p[a]) for a in p if p[a]>0])

    alphabet = list(p)
    f = [0]
    for a in p:
        f.append(f[-1]+p[a])
    f.pop()

    p = list(p.values())

    y.extend(precision*[0]) # dummy zeros to prevent index out of bound errors
    x = n*[0] # initialise all zeros

    # initialise by taking first 'precision' bits from y and converting to a number
    value = int(''.join(str(a) for a in y[0:precision]), 2)
    y_position = precision # position where currently reading y
    lo,hi = 0,one

    x_position = 0
    while 1:
        if x_position % 100 == 0:
            so.write('Arithmetic decoded %d%%    \r' % int(floor(x_position/n*100)))
            so.flush()

        lohi_range = hi - lo + 1
        a = bisect(f, (value-lo)/lohi_range) - 1
        x[x_position] = alphabet[a]

        lo = lo + int(ceil(f[a]*lohi_range))
        hi = lo + int(floor(p[a]*lohi_range))
        if (lo == hi):
            raise NameError('Zero interval!')

        while True:
            if hi < half:
                # do nothing
                pass
            elif lo >= half:
                lo = lo - half
                hi = hi - half
                value = value - half
            elif lo >= quarter and hi < threequarters:
                lo = lo - quarter
                hi = hi - quarter
                value = value - quarter
            else:
                break
            lo = 2*lo
            hi = 2*hi + 1
            value = 2*value + y[y_position]
            y_position += 1
            if y_position == len(y):
                break

        x_position += 1
        if x_position == n or y_position == len(y):
            break

    return(x)

We encode and decode Hamlet again using arithmetic coding and verify that the first few lines of the play look as expected.

In [45]:
arith_encoded = arith_encode(hamlet, p)
arith_decoded = arith_decode(arith_encoded, p, Nin)
print('\n'+''.join(arith_decoded[:294]))


        HAMLET


        DRAMATIS PERSONAE


CLAUDIUS        king of Denmark. (KING CLAUDIUS:)

HAMLET  son to the late, and nephew to the present king.

POLONIUS        lord chamberlain. (LORD POLONIUS:)

HORATIO friend to Hamlet.

LAERTES son to Polonius.

LUCIANUS        nephew to the king.


We now repeat the steps above but introduce a one bit mistake (bit 399 flipped) and observe the effect on the decoded text. Repeat this experiment varying the location of the mistake or adding more than one mistake. What do you observe? Can you explain why?

In [47]:
arith_corrupted = arith_encoded.copy()
arith_corrupted[399] ^= 1
arith_decoded = arith_decode(arith_corrupted, p, Nin)
print('\n'+''.join(arith_decoded[:294]))


        HAMLET


        DRAMATIS PERSONAE


CLAUDIUS        king of Denmark. I he   soeoW  Llon,o gn swt  s  .em urlo agdehnainf ts 
  aele 
 Nhhyr  a  Sh.ath rue oetantoboso nsiiho- eArpIntn      yd  qtr ayeth soon  he hh? Sesct aT  hLoT e n   Hysst 
o a gw sf  et deLtlucEir on  en  tawfiui 
